In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from Tools.leica_tools import parse_lif
from Tools.exp_tools import Experiment

In [11]:
expID = 'NKIP_FA_082'#
experiment = Experiment(expID)

In [ ]:
experiment.detect_droplets(model_name='yolo_v8_s_backbone_v3.keras')

In [ ]:
experiment.cell_count(model_name='cell_count_v3.h5')

# Legacy transfer

In [3]:
import glob
import pandas as pd


def get_df(path):
    df = pd.read_csv(path)
    df.rename(
        columns={'GlobalID': 'droplet_id', 'Effector': 'effector', 'Target': 'target', 'dead_Effector': 'dead_effector',
                 'dead_Target': 'dead_target'}, inplace=True)
    df = df.astype({'droplet_id': int})
    df.set_index('droplet_id', inplace=True)
    df.drop(columns=['i'], inplace=True)
    return df

In [44]:
exp_id = 'NKIP_FA_056'
experiment = Experiment(exp_id)

In [ ]:
# add droplets to db

droplets = pd.read_csv(os.path.join(os.getenv('EXP_DIR'), exp_id, 'droplets.csv'))
droplets.rename(columns={'GlobalID': 'droplet_id', 'frameID':'frame_id'}, inplace=True)
droplets['experiment_id'] = exp_id

experiment.db.add_droplets(droplets)
tfrecord_manifest = {
    'n_frames': droplets.index.size,
    'y_shape': 128,
    'x_shape': 128,
    'n_channels': experiment.handler.channel_df.index.size,
    'channel_info': experiment.handler.channel_df['channel_name'].to_dict(),
}
def frame_generator():
    for frame_id in experiment.frame_df.index:
        yield experiment.handler.get_frame(frame_id)
experiment.db.add_dataset(frame_generator(), tfrecord_manifest)

In [45]:
experiment.db.clear_annotations()

In [46]:
# add cell_count predictions to annotations table

droplets = pd.read_csv(os.path.join(os.getenv('EXP_DIR'), exp_id, 'droplets.csv'))
droplets.rename(columns={'GlobalID': 'droplet_id', 'frameID':'frame_id'}, inplace=True)
droplets['experiment_id'] = exp_id
droplets.set_index('droplet_id', inplace=True)
droplets.rename(columns={'SingleClass_dead_Effector': 'dead_effector',
                         'SingleClass_Target': 'target',
                         'SingleClass_Effector': 'effector',
                         'SingleClass_dead_Target': 'dead_target'}, inplace=True)
selected_annotations = ['dead_effector', 'dead_target', 'effector', 'target']
droplets[selected_annotations].head()

,dead_effector,dead_target,effector,target
droplet_id,,,,
0,1,1,0,0
1,0,0,0,2
2,0,0,0,1
3,0,0,0,0
4,0,0,0,1


In [47]:
source = 'cell_count_v2'
annotations = []

for droplet_id, droplet in droplets[selected_annotations].iterrows():
    for label_type, value  in droplet.items():
        annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': str(int(value)), 'source': source})

annotation_df = pd.DataFrame(annotations)
experiment.db.add_annotations(annotation_df)

In [48]:
# Import workpackages
wps = glob.glob(os.path.join(os.getenv('EXP_DIR'), exp_id, '*', 'WP_*.csv'))
annotations = []
for wp in wps:
    wp_id = os.path.basename(wp)
    df = get_df(wp).sort_values('droplet_id')

    for droplet_id, droplet in df.iterrows():
        for label_type, value in droplet.items():
            if not pd.isna(value):
                annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': str(int(value)), 'ap_id': wp_id})

annotation_df = pd.DataFrame(annotations)
experiment.db.add_annotations(annotation_df)